In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1,32,3,1)#in_channels,out_channels,kernel_size,stride
        self.conv2 = nn.Conv2d(32,64,3,1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216,128)#in_channels,out_channels
        self.fc2 = nn.Linear(128,10)
        
    def forward(self,x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x,2)
        x = self.dropout1(x)
        x = torch.flatten(x,1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x,dim=1)
        return output

In [16]:
def train(model,device,train_loader,optimizer,epoch,log_interval):
    model.train()#切换到训练状态
    for batch_idx, (data,target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output,target)#negative log likelihood, nll_loss+log_softmax = cross_entropy
        loss.backward()#反向计算，计算当前的梯度
        optimizer.step()#更新网络参数
        if batch_idx%log_interval == 0:
            print('Train Epoch:{}[{}/{}({:.0f}%)]\tLoss:{:.6f}'.format(epoch,batch_idx*len(data), len(train_loader.dataset),100.*batch_idx/len(train_loader),loss.item()))

In [4]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data,target in test_loader:
            data, target = data.to(device),target.to(device)
            output = model(data)
            test_loss += F.nll_loss(out,target,reduction = 'sum').item()
            pred = out.argmax(dim = 1, keepdim = True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    
    print('\nTest set: Average loss{:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss,correct,len(test_loader.dataset),100.*correct/len(test_loader.dataset)))

In [5]:
from torchvision import datasets,transforms

In [6]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,),(0.3081,))])#mean = 0.1307, std = 0.3081

In [7]:
dataset1 = datasets.MNIST('../data', train=True, download=True, transform = transform)
dataset2 = datasets.MNIST('../data', train=False, transform = transform)

In [9]:
train_loader = torch.utils.data.DataLoader(dataset1,batch_size = 10)
test_loader = torch.utils.data.DataLoader(dataset2,batch_size = 10)

In [14]:
torch.manual_seed(0)
device = torch.device("cpu")
epoch_size = 20
log_interval = 10

In [11]:
model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(),lr =1)

In [13]:
from torch.optim.lr_scheduler import StepLR
scheduler = StepLR(optimizer,step_size = 1, gamma = 0.7)
#learning_rate decay 每次调用这个函数，lr = lr*0.7

In [ ]:
for epoch in range(1,epoch_size+1):
    train(model,device,train_loader,optimizer,epoch,log_interval)
    test(model,device,test_loader)
    scheduler.step()

Train Epoch:1[0/60000(0%)]	Loss:0.004389
Train Epoch:1[100/60000(0%)]	Loss:0.014616
Train Epoch:1[200/60000(0%)]	Loss:0.007649
Train Epoch:1[300/60000(0%)]	Loss:0.002832
Train Epoch:1[400/60000(1%)]	Loss:0.076742
Train Epoch:1[500/60000(1%)]	Loss:1.076321
Train Epoch:1[600/60000(1%)]	Loss:0.013874
Train Epoch:1[700/60000(1%)]	Loss:0.391330
Train Epoch:1[800/60000(1%)]	Loss:0.004285
Train Epoch:1[900/60000(2%)]	Loss:0.979146
Train Epoch:1[1000/60000(2%)]	Loss:0.166000
Train Epoch:1[1100/60000(2%)]	Loss:0.008147
Train Epoch:1[1200/60000(2%)]	Loss:0.092703
Train Epoch:1[1300/60000(2%)]	Loss:0.001128
Train Epoch:1[1400/60000(2%)]	Loss:0.396945
Train Epoch:1[1500/60000(2%)]	Loss:0.006641
Train Epoch:1[1600/60000(3%)]	Loss:0.953900
Train Epoch:1[1700/60000(3%)]	Loss:0.000967
Train Epoch:1[1800/60000(3%)]	Loss:0.042953
Train Epoch:1[1900/60000(3%)]	Loss:0.002762
Train Epoch:1[2000/60000(3%)]	Loss:0.062273
Train Epoch:1[2100/60000(4%)]	Loss:0.008224
Train Epoch:1[2200/60000(4%)]	Loss:0.172791


Train Epoch:1[18200/60000(30%)]	Loss:0.016425
Train Epoch:1[18300/60000(30%)]	Loss:0.087723
Train Epoch:1[18400/60000(31%)]	Loss:0.001083
Train Epoch:1[18500/60000(31%)]	Loss:0.169559
Train Epoch:1[18600/60000(31%)]	Loss:0.078729
Train Epoch:1[18700/60000(31%)]	Loss:0.186130
Train Epoch:1[18800/60000(31%)]	Loss:0.002464
Train Epoch:1[18900/60000(32%)]	Loss:0.001252
Train Epoch:1[19000/60000(32%)]	Loss:0.000711
Train Epoch:1[19100/60000(32%)]	Loss:0.000718
Train Epoch:1[19200/60000(32%)]	Loss:0.003271
Train Epoch:1[19300/60000(32%)]	Loss:0.001058
Train Epoch:1[19400/60000(32%)]	Loss:0.003575
Train Epoch:1[19500/60000(32%)]	Loss:0.714999
Train Epoch:1[19600/60000(33%)]	Loss:0.021328
Train Epoch:1[19700/60000(33%)]	Loss:0.004773
Train Epoch:1[19800/60000(33%)]	Loss:0.114177
Train Epoch:1[19900/60000(33%)]	Loss:0.007633
Train Epoch:1[20000/60000(33%)]	Loss:0.008295
Train Epoch:1[20100/60000(34%)]	Loss:0.054827
Train Epoch:1[20200/60000(34%)]	Loss:0.043690
Train Epoch:1[20300/60000(34%)]	Lo